In [2]:
import sys
sys.path.append('../../')

import pandas as pd
import gadme
import os

In [5]:
print(os.getcwd())

/home/lukas/projects/Bird2Vec/notebooks


In [8]:
metadata = pd.read_csv('../gadme/data/xeno-canto/na_metadata.csv', index_col='id')

In [9]:
metadata.head(3)

,lat,lng,type,sex,primary,continent,secondary,file_name,ebird_code
id,,,,,,,,,
564620,9.1332,-79.7195,song,NaN,tinamus major,North America,"['Patagioenas nigrirostris', 'Rhynchocyclus ol...",data/xeno-canto/North America/Tinamus major/XC...,gretin1
411236,10.4295,-84.0095,song,NaN,tinamus major,North America,[],data/xeno-canto/North America/Tinamus major/XC...,gretin1
363937,9.1332,-79.7195,song,NaN,tinamus major,North America,"['Cyphorhinus phaeocephalus', 'Formicarius ana...",data/xeno-canto/North America/Tinamus major/XC...,gretin1


In [11]:
metadata.shape

(83089, 9)

In [16]:
metadata["file_name"].iloc[0]

'data/xeno-canto/North America/Tinamus major/XC564620.mp3'

In [12]:
subset_na = metadata.sample(1000)

In [13]:
subset_na.shape

(1000, 9)

In [6]:
from gadme.data.utils import create_hg_dataset

In [8]:
hg_dataset = create_hg_dataset(metadata, min_occ=20)

In [9]:
hg_dataset

Dataset({
    features: ['lat', 'lng', 'type', 'sex', 'primary', 'continent', 'secondary', 'file_name', 'ebird_code', 'id'],
    num_rows: 78603
})

In [10]:
hg_dataset[0]

{'lat': 9.1332,
 'lng': -79.7195,
 'type': 'song',
 'sex': None,
 'primary': 784,
 'continent': 'North America',
 'secondary': "['Patagioenas nigrirostris', 'Rhynchocyclus olivaceus', 'Trogon caligatus', 'Xiphorhynchus susurrans', 'Vireolanius pulchellus', 'Amazona autumnalis']",
 'file_name': 'data/xeno-canto/North America/Tinamus major/XC564620.mp3',
 'ebird_code': 'gretin1',
 'id': 564620}

In [41]:
metadata = pd.read_csv('../gadme/data/xeno-canto/na_metadata_subset1k.csv', index_col='id')

In [42]:
metadata["file_name"].iloc[0]

'data/xeno-canto/North America/Campylorhynchus brunneicapillus/XC241017.mp3'

In [43]:
metadata['file_name'] = metadata['file_name'].str.replace(r'North America/.*?/', 'NA_subset1k/', regex=True)

In [44]:
metadata['file_name'] = '/home/lukas/projects/Bird2Vec/gadme/' + metadata['file_name']

In [45]:
metadata["file_name"].iloc[5]

'/home/lukas/projects/Bird2Vec/gadme/data/xeno-canto/NA_subset1k/XC186820.mp3'

In [46]:
from datasets import Dataset, Audio, Features

In [47]:
hf_ds = Dataset.from_pandas(metadata)

In [33]:
hf_ds

Dataset({
    features: ['lat', 'lng', 'type', 'sex', 'primary', 'continent', 'secondary', 'file_name', 'ebird_code', 'id'],
    num_rows: 1000
})

In [34]:
hf_ds.features

{'lat': Value(dtype='float64', id=None),
 'lng': Value(dtype='float64', id=None),
 'type': Value(dtype='string', id=None),
 'sex': Value(dtype='string', id=None),
 'primary': Value(dtype='string', id=None),
 'continent': Value(dtype='string', id=None),
 'secondary': Value(dtype='string', id=None),
 'file_name': Value(dtype='string', id=None),
 'ebird_code': Value(dtype='string', id=None),
 'id': Value(dtype='int64', id=None)}

In [48]:
hf_ds = hf_ds.rename_column("file_name", "audio")

In [63]:
hf_ds = hf_ds.cast_column(
    column="audio", 
    feature= Audio(
            sampling_rate=16_000,
            mono=True,
            decode=True
    )
)

In [64]:
import os
os.getcwd()

'/home/lukas/projects/Bird2Vec/notebooks'

In [69]:
sample = hf_ds[100]["audio"]
sample

{'path': '/home/lukas/projects/Bird2Vec/gadme/data/xeno-canto/NA_subset1k/XC690973.mp3',
 'array': array([ 0.00262681,  0.00254007, -0.00276999, ..., -0.01107635,
         0.00331435,  0.00253535]),
 'sampling_rate': 16000}

In [70]:
from IPython.display import Audio
Audio(data=sample["array"], rate=16_000)

In [71]:
hf_ds.push_to_hub(
    repo_id=
)

{'path': '/home/lukas/projects/Bird2Vec/gadme/data/xeno-canto/NA_subset1k/XC241017.mp3',
 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.55924442e-06, -2.74799049e-06,  0.00000000e+00]),
 'sampling_rate': 16000}

With decode=False, the Audio type simply gives you the path or the bytes of the audio file, without decoding it into an array,

decoding is done via librosa?: 
https://github.com/huggingface/datasets/blob/2.13.1/gadme/datasets/features/audio.py#L23